# Sparse Submanifold Autoencoders

### Run this notebook inside a directory that contains dlp_opendata_api folder

In [1]:
# Import Dependencies

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#%matplotlib notebook

import sys
sys.path.append("dlp_opendata_api")
from osf.image_api import image_reader_3d
from osf.particle_api import *
from osf.cluster_api import *

from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sparseconvnet as scn
import glob
import os.path as osp
import numpy as np

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

Welcome to JupyROOT 6.14/04


In [2]:
#ls /gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10
use_cuda = torch.cuda.is_available()

### Check if CUDA is working (GPU)

In [3]:
print(use_cuda)

True


In [51]:
class ClusteringAEData(Dataset):
    """
    A customized data loader for clustering.
    """
    def __init__(self, root, numPixels=192, filenames=None):
        """
        Initialize Clustering Dataset

        Inputs:
            - root: root directory of dataset
            - preload: if preload dataset into memory.
        """
        self.cluster_filenames = []
        self.energy_filenames = []
        self.root = root
        self.numPixels = str(numPixels)
        
        if filenames:
            self.energy_filenames = filenames[0]
            self.cluster_filenames = filenames[1]
            print(self.energy_filenames)

        self.energy_filenames.sort()
        self.cluster_filenames.sort()
        self.cluster_reader = cluster_reader(*self.cluster_filenames)
        self.energy_reader = image_reader_3d(*self.energy_filenames)
        self.len = self.energy_reader.entry_count()
        assert self.len == self.cluster_reader.entry_count()

    def __getitem__(self, index):
        """
        Get a sample from dataset.
        """
        voxel, label = self.cluster_reader.get_image(index)
        _, energy, _ = self.energy_reader.get_image(index)
        return (voxel, energy), label

    def __len__(self):
        """
        Total number of sampels in dataset.
        """
        return self.len

In [52]:
def ae_collate(batch):
    """
    Custom collate_fn for Autoencoder.
    """
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    return [data, target]

### Get Train, Dev, and Test Set

In [53]:
root = '/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10' #replace with your own path to root folder. 
trainset_cluster = [root + '/cluster/dlprod_cluster_192px_0{}.root'.format(i) for i in range(8)]
devset_cluster = [root + '/cluster/dlprod_cluster_192px_0{}.root'.format(8)]
#testset_cluster = [root + '/cluster/dlprod_cluster_192px_0{}.root'.format(9)]

trainset_energy = [root + '/dlprod_192px_0{}.root'.format(i) for i in range(8)]
devset_energy = [root + '/dlprod_192px_0{}.root'.format(8)]
#testset_energy = [root + '/dlprod_192px_0{}.root'.format(9)]

for i, f in enumerate(trainset_cluster):
    print(f)
    print(trainset_energy[i])
    
for i, f in enumerate(devset_cluster):
    print(f)
    print(devset_energy[i])
    
#for i, f in enumerate(testset_cluster):
#    print(f)
#    print(testset_energy[i])

trainset = ClusteringAEData(root, 192, filenames=[trainset_energy, trainset_cluster])
devset = ClusteringAEData(root, 192, filenames=[devset_energy, devset_cluster])
#testset = ClusteringAEData(root, 192, filenames=[testset_energy, testset_cluster])
print('Number of entries in training set: {}'.format(len(trainset)))
print('Number of entries in validation set: {}'.format(len(devset)))
#print('Number of entries in test set: {}'.format(len(testset)))

/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/cluster/dlprod_cluster_192px_00.root
/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/dlprod_192px_00.root
/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/cluster/dlprod_cluster_192px_01.root
/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/dlprod_192px_01.root
/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/cluster/dlprod_cluster_192px_02.root
/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/dlprod_192px_02.root
/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/cluster/dlprod_cluster_192px_03.root
/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/dlprod_192px_03.root
/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/cluster/dlprod_cluster_192px_04.root
/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/dlprod_192px_04.root
/gpfs/slac/staas/fs1/g/neutrino/kterao/data/dlprod_ppn_v10/cluster/dlprod_cluster_192px_05.root
/gpfs/slac/staas/fs1/g/n

In [54]:
trainloader = DataLoader(trainset, batch_size=1, shuffle=True, collate_fn=ae_collate, num_workers=1, pin_memory=False)
devloader = DataLoader(devset, batch_size=1, shuffle=True, collate_fn=ae_collate, num_workers=1, pin_memory=False)

In [56]:
entry, labels = trainset[48]
coords, energy = entry
# coords refer to coordinates of each pixel
print(coords)
# energy refer to pixel energy values
print(energy)
# labels refer to cluster labels
print(labels)
print("How many distinct clusters: {}".format(np.unique(labels)))

[[109  60   0]
 [110  60   0]
 [111  60   0]
 ...
 [ 38 102 191]
 [ 39 102 191]
 [ 40 102 191]]
[0.01040413 0.04662808 0.01040413 ... 0.03000904 0.11896198 0.03734245]
[9. 9. 9. ... 1. 1. 1.]
How many distinct clusters: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 17. 18. 24. 42.]


## Visualize Dataset

In [57]:
from localutil.data import *
from localutil.visualization import *
from plotly.offline import init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
#import plotly.io as pio

In [58]:
# Code by Brad Nelson, I redefine the localutil.visualization modules 
# for better pictures (I set size of markers = 1)

def scatter_energy(img_reader, n, threshold=0.0):
    """
    Creates graph object for energy scatter plot
    Args:
        img_reader (image_reader_3d)
        n (int): image to plot
        threshold (optional): threshold energies below a certain value
    """
    voxels, energy, labels = img_reader.get_image(n)
    inds = energy > threshold
    trace = go.Scatter3d(x=voxels[inds,0], y=voxels[inds,1], z=voxels[inds,2],
                    mode='markers',
                    marker = dict(
                        size = 1,
                        color = np.log(energy[inds]),
                        colorscale='Viridis',
                        opacity=0.8
                    ), hovertext=energy)
    return trace

def scatter_classes(img_reader, n):
    voxels, types = img_reader.get_image(n)
    trace = go.Scatter3d(x=voxels[:,0], y=voxels[:,1], z=voxels[:,2],
                    mode='markers',
                    marker = dict(
                        size = 1,
                        color = types,
                        colorscale='Viridis',
                        opacity=0.8
                    ), hovertext=types)
    return trace

def scatter_clusters(voxels, clusters):
    trace = go.Scatter3d(x=voxels[:,0], y=voxels[:,1], z=voxels[:,2],
                    mode='markers',
                    marker = dict(
                        size = 1,
                        color = clusters,
                        colorscale='Viridis',
                        opacity=0.8
                    ), hovertext=clusters)
    return trace

In [59]:
eventn = 48
trace1 = scatter_classes(trainset.cluster_reader, eventn)
trace2 = scatter_energy(trainset.energy_reader, eventn)
fig = go.Figure(data=[trace1])
iplot(fig)
#pio.write_image(fig, 'images/fig1.pdf')

In [60]:
print(coords)
print(energy)
print(labels)

[[109  60   0]
 [110  60   0]
 [111  60   0]
 ...
 [ 38 102 191]
 [ 39 102 191]
 [ 40 102 191]]
[0.01040413 0.04662808 0.01040413 ... 0.03000904 0.11896198 0.03734245]
[9. 9. 9. ... 1. 1. 1.]


In [61]:
import pandas as pd
labels_plotting = labels.astype(int)
labels_plotting = pd.Series(labels_plotting)
labels_plotting

0        9
1        9
2        9
3        9
4        9
5        9
6        9
7        9
8        9
9        8
10       9
11       9
12       9
13       9
14       9
15       9
16       9
17       9
18       9
19       9
20       9
21       9
22       9
23       9
24       9
25       9
26       9
27       9
28       9
29       9
        ..
5150    42
5151    42
5152    42
5153    42
5154    42
5155    42
5156    42
5157    42
5158    42
5159    42
5160    42
5161    42
5162    42
5163    42
5164    42
5165    42
5166    42
5167    42
5168    42
5169    42
5170     1
5171     1
5172     1
5173     1
5174     1
5175     1
5176     1
5177     1
5178     1
5179     1
Length: 5180, dtype: int64

In [62]:
colorsIdx = {0: 'rgb(31, 119, 180)', 1: 'rgb(255, 127, 14)',
             2: 'rgb(44, 160, 44)', 3: 'rgb(214, 39, 40)',
             4: 'rgb(148, 103, 189)', 5:'rgb(140, 86, 75)',
             6: 'rgb(227, 119, 194)', 7: 'rgb(127, 127, 127)',
             8: 'rgb(188, 189, 34)', 9: 'rgb(23, 190, 207)',
             17: 'rgb(255,234,0)', 18: 'rgb(255,111,0)',
             24: 'rgb(150,0,90)', 42: 'rgb(0,0,200)'}
cols      = labels_plotting.map(colorsIdx)

trace = go.Scatter3d(x=coords[:,0], y=coords[:,1], z=coords[:,2],
                    mode='markers',
                    marker = dict(
                        size = 1,
                        color = cols,
                        opacity=0.8
                    ), hovertext=labels_plotting)
fig = go.Figure(data=[trace])
iplot(fig)

Here, each row of 'coords' is the x,y,z coordinates of the active pixels of the 3D tensor, and each row of 'y' is the value of the pixel at the corresponding 'coords' coordinates. That is, at (97, 42, 5) we have pixel value 0.0156. This notation is called a Sparse Representation since it only gives the active pixel sites for a large sparse matrix. Here, there are only two clusters, since np.unique(labels) gives 0 and 2, which are the integer labels of the clusters. We have one label per pixel site. 

## TODO: 
### 1. Make 3D Visualization for Sparse Matrices
### 2. Run DBSCAN on each cluster example and see how it performs. 

## Get Pretrained UResNet Module

In [63]:
class UResNet(torch.nn.Module):
    def __init__(self, dim=3, size=192, nFeatures=16, depth=5, nClasses=5):
        import sparseconvnet as scn
        super(UResNet, self).__init__()
        #self._flags = flags
        dimension = dim
        reps = 2  # Conv block repetition factor
        kernel_size = 2  # Use input_spatial_size method for other values?
        m = nFeatures  # Unet number of features
        nPlanes = [i*m for i in range(1, depth+1)]  # UNet number of features per level
        # nPlanes = [(2**i) * m for i in range(1, num_strides+1)]  # UNet number of features per level
        nInputFeatures = 1
        self.sparseModel = scn.Sequential().add(
           scn.InputLayer(dimension, size, mode=3)).add(
           scn.SubmanifoldConvolution(dimension, nInputFeatures, m, 3, False)).add( # Kernel size 3, no bias
           scn.UNet(dimension, reps, nPlanes, residual_blocks=True, downsample=[kernel_size, 2])).add(  # downsample = [filter size, filter stride]
           scn.BatchNormReLU(m)).add(
           scn.OutputLayer(dimension))
        self.linear = torch.nn.Linear(m, nClasses)

    def forward(self, point_cloud):
        """
        point_cloud is a list of length minibatch size (assumes mbs = 1)
        point_cloud[0] has 3 spatial coordinates + 1 batch coordinate + 1 feature
        shape of point_cloud[0] = (N, 4)
        """
        coords = point_cloud[:, 0:-1].float()
        features = point_cloud[:, -1][:, None].float()
        x = self.sparseModel((coords, features))
        x = self.linear(x)
        return [x]

In [64]:
def get_unet(fname, dimension=3, size=192, nFeatures=16, depth=5, nClasses=5):
    model = UResNet(dim=dimension, size=size, nFeatures=nFeatures, depth=depth, nClasses=nClasses)
    model = nn.DataParallel(model)
    #print(model.state_dict().keys())
    checkpoint = torch.load(fname, map_location='cpu')
    #print()
    #print(checkpoint['state_dict'].keys())
    model.load_state_dict(checkpoint['state_dict'], strict=True)
    # just return the pre-trained unet
    return model.module.sparseModel

In [65]:
fname = '/gpfs/slac/staas/fs1/g/neutrino/.scn_paper/new/sparse_is192_uns5_uf16_bs64/weights3/snapshot-29999.ckpt'
unet = get_unet(fname)
unet.eval()

Sequential(
  (0): InputLayer()
  (1): SubmanifoldConvolution 1->16 C3
  (2): Sequential(
    (0): ConcatTable(
      (0): Identity()
      (1): Sequential(
        (0): BatchNormReLU(16,eps=0.0001,momentum=0.9,affine=True)
        (1): SubmanifoldConvolution 16->16 C3
        (2): BatchNormReLU(16,eps=0.0001,momentum=0.9,affine=True)
        (3): SubmanifoldConvolution 16->16 C3
      )
    )
    (1): AddTable()
    (2): ConcatTable(
      (0): Identity()
      (1): Sequential(
        (0): BatchNormReLU(16,eps=0.0001,momentum=0.9,affine=True)
        (1): SubmanifoldConvolution 16->16 C3
        (2): BatchNormReLU(16,eps=0.0001,momentum=0.9,affine=True)
        (3): SubmanifoldConvolution 16->16 C3
      )
    )
    (3): AddTable()
    (4): ConcatTable(
      (0): Identity()
      (1): Sequential(
        (0): BatchNormReLU(16,eps=0.0001,momentum=0.9,affine=True)
        (1): Convolution 16->32 C2/2
        (2): Sequential(
          (0): ConcatTable(
            (0): Identity()
    

In [66]:
class AdjacencyMLP(nn.Module):
    def __init__(self, input_dim=38, nHidden1=200, nHidden2=200, nClasses=2):
        super(AdjacencyMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, nHidden1)
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(nHidden1, nHidden2)
        nn.init.kaiming_normal_(self.fc2.weight)
        self.fc3 = nn.Linear(nHidden2, nClasses)
        nn.init.kaiming_normal_(self.fc3.weight)
        
        self.bn_1 = nn.BatchNorm1d(nHidden1)
        self.bn_2 = nn.BatchNorm1d(nHidden2)
        
    def forward(self, x):
        x = F.leaky_relu(self.bn_1(self.fc1(x)))
        x = F.leaky_relu(self.bn_2(self.fc2(x)))
        x = self.fc3(x)
        return x

In [67]:
model = AdjacencyMLP()

In [68]:
class MLPDataset(Dataset):
    
    def __init__(self, labels, data, use_cuda=False):
        
        self.labels = labels
        self.encoding = data
        
    def __getitem__(self, index):
        return self.encoding[index, :], self.labels[index]
    
    def __len__(self):
        return self.encoding.shape[0]

In [89]:
def train_MLP(model, loader, optimizer, epochs, use_cuda=False, log=2):
    """
    Function for training SparseUResNet.
    """
    for ep in range(epochs):
        for _, entry in enumerate(loader):
            data, label = entry
            optimizer.zero_grad()
            batch_len = label.shape[0]
            idx = torch.randperm(batch_len)
            X1 = data
            y1 = label
            X2 = data[idx]
            y2 = label[idx]
            adjacency = (y1 == y2)
            adjacency = (y1 == y2).to(device='cpu', dtype=torch.long)
            X = torch.cat([X1, X2], dim=1)
            #print(X)
            scores = model.forward(X)
            pred = torch.argmax(F.log_softmax(scores, dim=1), dim=1)
            print(scores)
            print(pred)
            loss = F.cross_entropy(scores, adjacency)
            loss.backward()
            optimizer.step()
            accuracy = pred == adjacency
            accuracy = accuracy.to(device='cpu', dtype=torch.long)
            #print(len(accuracy))
            accuracy = float(torch.sum(accuracy)) / len(accuracy)
            #print(accuracy)
    return loss, accuracy

In [70]:
# Create Example Data
x = torch.LongTensor(coords)
y = torch.FloatTensor(energy.T)
y = y.view(-1, 1)
print(x)
print(y)
print(x.shape)
print(y.shape)
x = x.cuda()
y = y.cuda()

tensor([[109,  60,   0],
        [110,  60,   0],
        [111,  60,   0],
        ...,
        [ 38, 102, 191],
        [ 39, 102, 191],
        [ 40, 102, 191]])
tensor([[0.0104],
        [0.0466],
        [0.0104],
        ...,
        [0.0300],
        [0.1190],
        [0.0373]])
torch.Size([5180, 3])
torch.Size([5180, 1])


In [71]:
model = AdjacencyMLP()
with torch.no_grad():
    out = unet((x,y))
    out = torch.cat([x.float(), out], dim=1)
    print(out.shape)
out = out.to(device='cpu')
dataset = MLPDataset(labels, out)
train_loader_img = DataLoader(dataset, batch_size=3827, shuffle=True, num_workers=1, pin_memory=False)
#print(dataset.encoding.shape)
#print(len(dataset))

#train_MLP(model, train_loader_img, optimizer, 100, log=1)

torch.Size([5180, 19])


In [72]:
def save_checkpoint(checkpoint_path, model, optimizer):
    # state_dict: a Python dictionary object that:
    # - for a model, maps each layer to its parameter tensor;
    # - for an optimizer, contains info about the optimizer’s states and hyperparameters used.
    state = {
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)

In [73]:
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)

In [90]:
def train_whole(model, imgLoader, epoch, optimizer, log=1, save_interval=400):
    model.train()
    iteration = 0
    val_iteration = 0
    loss_hist = 'log/loss.csv'
    acc_hist = 'log/acc.csv'
    loss_hist = open(loss_hist, 'w')
    acc_hist = open(acc_hist, 'w')
    for ep in range(epoch):
        for i, img_batch in enumerate(imgLoader):
            data, labels = img_batch
            coords, values = data[0]
            coords, values, labels = torch.LongTensor(coords), torch.FloatTensor(values), torch.LongTensor(labels)
            coords, values = coords.cuda(), values.cuda()
            values = values.view(-1, 1)
            labels = labels.view(-1, 1)
            print(coords, coords.shape)
            print(values, values.shape)
            print(labels, labels.shape)
            with torch.no_grad():
                out = unet((coords, values))
                vec = torch.cat([coords.float(), out], dim=1)
            vec = vec.to(device='cpu')
            pixels = MLPDataset(labels, vec)
            pixel_batch_size = len(pixels)
            sub_trainLoader = DataLoader(pixels, batch_size=pixel_batch_size, 
                                         shuffle=True, num_workers=1, pin_memory=False)
            loss, accuracy = train_MLP(model, sub_trainLoader, optimizer, 1, log=1)
            if iteration % log == 0:
                print("Validation Sample = {}".format(val_iteration))
                dev_data, dev_labels = devset[val_iteration]
                print(dev_data, dev_labels)
                dev_labels = torch.from_numpy(dev_labels)
                coords, values = dev_data
                coords, values = coords.cuda(), values.cuda()
                with torch.no_grad():
                    #print(coords)
                    #print(values)
                    out = unet((coords, values))
                    vec = torch.cat([coords.float(), out], dim=1)
                    vec = vec.to(device='cpu')
                    idx = torch.randperm(vec.shape[0])
                    vec2 = vec[idx]
                    dev_labels2 = dev_labels[idx]
                    data = torch.cat([vec, vec2], dim=1)
                    data = data.to(device='cpu')
                    scores = model.forward(data)
                    pred = torch.argmax(F.log_softmax(scores, dim=1), dim=1)
                    truth = (dev_labels == dev_labels2).to(device='cpu', dtype=torch.long)
                    val_loss = F.cross_entropy(scores, truth)
                    correct = (pred == truth).to(device='cpu', dtype=torch.long)
                    #print(correct.shape)
                    val_acc = float(torch.sum(correct)) / len(correct)
                    #print(val_acc)
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]'.format(
                        ep, i, len(imgLoader.dataset),
                        100. * float(i) / len(imgLoader)))
                    print("Training Accuracy = {}\tLoss: {:.6f}".format(accuracy, loss))
                    print("Validation Accuracy = {}\tLoss: {:.6f}\n".format(val_acc, val_loss))
                    loss = float(loss)
                    loss_hist.write('{}, {}\n'.format(loss, val_loss))
                    acc_hist.write('{}, {}\n'.format(accuracy, val_acc))
                val_iteration += 1
            if iteration % save_interval == 0:
                checkpoint_path = 'checkpoint_{}.pt'.format(iteration)
                save_checkpoint(checkpoint_path, model, optimizer)
            iteration += 1

In [91]:
model = AdjacencyMLP()

p = {}
p['n_epochs'] = 100
p['initial_lr'] = 0.01
p['lr_decay'] = 4e-3
p['weight_decay'] = 1e-5
p['momentum'] = 0.9
p['check_point'] = False
optimizer = optim.SGD(model.parameters(),
    lr=p['initial_lr'],
    momentum=p['momentum'],
    weight_decay=p['weight_decay'],
    nesterov=True)

#model = AdjacencyMLP()
#load_checkpoint('checkpoints/checkpoint_39600.pt', model, optimizer)

losses, accuracies = train_whole(model, trainloader, 1, optimizer, log=1)

(tensor([[ 94,  19,   0],
        [ 95,  19,   0],
        [ 96,  19,   0],
        ...,
        [128,  54, 159],
        [129,  54, 159],
        [130,  54, 159]], device='cuda:0'), torch.Size([4911, 3]))
(tensor([[0.0490],
        [0.0996],
        [0.0270],
        ...,
        [0.0971],
        [0.4492],
        [0.0971]], device='cuda:0'), torch.Size([4911, 1]))
(tensor([[25],
        [25],
        [25],
        ...,
        [20],
        [20],
        [20]]), torch.Size([4911, 1]))
tensor([[ 1.2322, -0.2200],
        [ 0.1301,  0.0294],
        [ 0.2765, -0.4376],
        ...,
        [ 0.3495, -0.1767],
        [ 1.2006, -0.1544],
        [ 1.9129,  0.0327]], grad_fn=<ThAddmmBackward>)
tensor([0, 0, 0,  ..., 0, 0, 0])


RuntimeError: multi-target not supported at /pytorch/aten/src/THNN/generic/ClassNLLCriterion.c:21

In [86]:
devset[0]

((array([[114,  69,   0],
         [115,  69,   0],
         [116,  69,   0],
         ...,
         [ 32, 113, 189],
         [ 30, 114, 189],
         [ 31, 114, 189]], dtype=int32),
  array([0.05206477, 0.0566299 , 0.05070747, ..., 0.01099252, 0.01121151,
         0.01323677], dtype=float32)),
 array([31., 31., 31., ..., 34., 34., 34.], dtype=float32))

In [ ]:
f = open('data.csv', 'w')
f.write('asdasdassasddsad\n')
f.close()